In [1]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [2]:
def black_scholes(S, K, T, r, sigma, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == "call":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Type d'option non reconnu. Utilisez 'call' ou 'put'.")
    
    return price


S = 100         
K = 100       
T = 1          
r = 0.05      
sigma = 0.2    

# Listes de valeurs pour chaque paramètre
S_values = np.linspace(50, 150, 100)      # Prix sous-jacent
K_values = np.linspace(80, 120, 100)      # Prix d'exercice
T_values = np.linspace(0.1, 2, 100)       # Maturité
r_values = np.linspace(0.01, 0.1, 100)    # Taux sans risque
sigma_values = np.linspace(0.1, 0.5, 100) # Volatilité

# Graphiques de l'impact de chaque paramètre pour les options Put
plt.figure(figsize=(15, 10))

# Effet du prix du sous-jacent S
prices_S_p = [black_scholes(S, K, T, r, sigma, option_type="put") for S in S_values]
prices_S_c = [black_scholes(S, K, T, r, sigma, option_type="put") for S in S_values]
plt.subplot(2, 3, 1)
plt.plot(S_values, prices_S_c, label="Call Price", color="orange")
plt.plot(S_values, prices_S_p, label="Put Price", color="blue")
plt.xlabel("Prix de l'actif sous-jacent S")
plt.ylabel("Prix de l'option")
plt.title("Impact de S sur le prix de l'option Put")
plt.grid(True)

# Effet du prix d'exercice K
prices_K_p = [black_scholes(S, K, T, r, sigma, option_type="put") for K in K_values]
prices_K_c= [black_scholes(S, K, T, r, sigma, option_type="call") for K in K_values]
plt.subplot(2, 3, 2)
plt.plot(K_values, prices_K_p, label="Put Price", color="orange")
plt.plot(K_values, prices_K_c, label="Put Price", color="blue")
plt.xlabel("Prix d'exercice K")
plt.ylabel("Prix de l'option")
plt.title("Impact de K sur le prix de l'option Put")
plt.grid(True)

# Effet de la maturité T
prices_T = [black_scholes(S, K, T, r, sigma, option_type="put") for T in T_values]
plt.subplot(2, 3, 3)
plt.plot(T_values, prices_T, label="Put Price", color="green")
plt.xlabel("Maturité T (en années)")
plt.ylabel("Prix de l'option")
plt.title("Impact de T sur le prix de l'option Put")
plt.grid(True)

# Effet du taux sans risque r
prices_r = [black_scholes(S, K, T, r, sigma, option_type="put") for r in r_values]
plt.subplot(2, 3, 4)
plt.plot(r_values, prices_r, label="Put Price", color="red")
plt.xlabel("Taux sans risque r")
plt.ylabel("Prix de l'option")
plt.title("Impact de r sur le prix de l'option Put")
plt.grid(True)

# Effet de la volatilité sigma
prices_sigma = [black_scholes(S, K, T, r, sigma, option_type="put") for sigma in sigma_values]
plt.subplot(2, 3, 5)
plt.plot(sigma_values, prices_sigma, label="Put Price", color="purple")
plt.xlabel("Volatilité sigma")
plt.ylabel("Prix de l'option")
plt.title("Impact de sigma sur le prix de l'option Put")
plt.grid(True)

plt.tight_layout()
plt.show()


NameError: name 'np' is not defined

In [6]:
import numpy as np
from scipy.stats import norm

def black_scholes_greeks(S, K, T, r, sigma, option_type="call"):
    """
    Calcule les grecs d'une option européenne en utilisant le modèle Black-Scholes.
    
    :param S: float - Prix actuel de l'actif sous-jacent
    :param K: float - Prix d'exercice de l'option
    :param T: float - Temps restant jusqu'à l'échéance (en années)
    :param r: float - Taux d'intérêt sans risque
    :param sigma: float - Volatilité du sous-jacent
    :param option_type: str - "call" pour un call, "put" pour un put
    :return: dict - Valeurs des grecs
    """
    # Calcul de d1 et d2
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Delta
    if option_type == "call":
        delta = norm.cdf(d1)
    elif option_type == "put":
        delta = -norm.cdf(-d1)
    else:
        raise ValueError("Type d'option non reconnu. Utilisez 'call' ou 'put'.")

    # Gamma (identique pour call et put)
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    
    # Vega (identique pour call et put)
    vega = S * norm.pdf(d1) * np.sqrt(T)
    
    # Theta
    if option_type == "call":
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) 
                 - r * K * np.exp(-r * T) * norm.cdf(d2))
    elif option_type == "put":
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) 
                 + r * K * np.exp(-r * T) * norm.cdf(-d2))
    
    # Rho
    if option_type == "call":
        rho = K * T * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2)
    
    # Vomma (dérivée seconde de Vega par rapport à la volatilité)
    vomma = vega * d1 * d2 / sigma

    return {
        "Delta": delta,
        "Gamma": gamma,
        "Vega": vega,
        "Theta": theta,
        "Rho": rho,
        "Vomma": vomma
    }

# Exemple d'utilisation
S = 100      # Prix de l'actif sous-jacent
K = 100      # Prix d'exercice
T = 1        # Maturité en années
r = 0.05     # Taux sans risque
sigma = 0.2  # Volatilité

# Calcul des grecs pour un call et un put
greeks_call = black_scholes_greeks(S, K, T, r, sigma, option_type="call")
greeks_put = black_scholes_greeks(S, K, T, r, sigma, option_type="put")

print("Grecs pour l'option Call:", greeks_call)
print("Grecs pour l'option Put:", greeks_put)


Grecs pour l'option Call: {'Delta': 0.6368306511756191, 'Gamma': 0.018762017345846895, 'Vega': 37.52403469169379, 'Theta': -6.414027546438197, 'Rho': 53.232481545376345, 'Vomma': 9.850059106569622}
Grecs pour l'option Put: {'Delta': -0.3631693488243809, 'Gamma': 0.018762017345846895, 'Vega': 37.52403469169379, 'Theta': -1.657880423934626, 'Rho': -41.89046090469506, 'Vomma': 9.850059106569622}
